In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from prophet import Prophet
import plotly.express as px
import plotly.figure_factory as ff

import warnings
from soupsieve import select
warnings.filterwarnings('ignore')
1
# Shows all columns 
pd.set_option("display.max_columns", None)

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
data = pd.read_excel("Data_seanality_forecast_stock.xlsx")

In [3]:
data.shape

(2160, 9)

In [4]:
data.head()

,Date,Location,Item,Quantity,Amount,Trend,seasonality,remainder,Forecasted_Quantity
0,2022-03-10,FREMONT,CHICKEN XXXXX - O,31.71118,189.95000,31.71118,1,1,79.18177
1,2022-03-11,FREMONT,CHICKEN XXXXX - O,53.33056,319.45000,53.33056,1,1,85.55376
2,2022-03-15,RICHMOND CA,FILLET QQQQ,94.19899,752.65000,94.19899,1,1,82.20319
3,2022-03-16,FREMONT,CHICKEN XXXXX - O,84.58932,506.69000,84.58932,1,1,80.07380
4,2022-03-16,RICHMOND CA,FILLET QQQQ,59.15895,472.68000,59.15895,1,1,80.48358


In [5]:
data["stock_percentage"] = ((data["Forecasted_Quantity"]-data["Quantity"])/data["Quantity"])*100

In [6]:
data.head()

,Date,Location,Item,Quantity,Amount,Trend,seasonality,remainder,Forecasted_Quantity,stock_percentage
0,2022-03-10,FREMONT,CHICKEN XXXXX - O,31.71118,189.95000,31.71118,1,1,79.18177,149.69670
1,2022-03-11,FREMONT,CHICKEN XXXXX - O,53.33056,319.45000,53.33056,1,1,85.55376,60.42164
2,2022-03-15,RICHMOND CA,FILLET QQQQ,94.19899,752.65000,94.19899,1,1,82.20319,-12.73453
3,2022-03-16,FREMONT,CHICKEN XXXXX - O,84.58932,506.69000,84.58932,1,1,80.07380,-5.33817
4,2022-03-16,RICHMOND CA,FILLET QQQQ,59.15895,472.68000,59.15895,1,1,80.48358,36.04633


In [7]:
conditions = [
    (data['stock_percentage'] <= 0),
    (data['stock_percentage'] > 0) & (data['stock_percentage'] <= 25),
    (data['stock_percentage'] > 25)]

In [8]:
values = ['Understock', 'Onhand', 'Overstock']

In [9]:
data["stock"]=np.select(conditions, values)

In [10]:
data.sample(10)

,Date,Location,Item,Quantity,Amount,Trend,seasonality,remainder,Forecasted_Quantity,stock_percentage,stock
143,2022-04-07,HAYWARD,DRUMSTICKS NNNN,562.05051,556.43000,562.05051,1,1,556.43903,-0.99839,Understock
279,2022-04-22,FREMONT,CHICKEN XXXXX - O,16.50084,98.84000,16.50084,1,1,69.29311,319.93689,Overstock
1843,2022-10-21,HAYWARD,FILLET QQQQ,68.48053,615.64000,68.48053,1,1,62.39642,-8.88444,Understock
1674,2022-10-03,LAKEWOOD,FILLET QQQQ,21.13125,189.97000,21.13125,1,1,43.25648,104.70386,Overstock
177,2022-04-12,HAYWARD,DRUMSTICKS NNNN,592.74742,586.82000,592.74742,1,1,621.88879,4.91632,Onhand
287,2022-04-25,RICHMOND CA,CHICKEN XXXXX - O,20.67111,123.82000,20.67111,1,1,78.94921,281.93020,Overstock
756,2022-06-21,HAYWARD,CHICKEN XXXXX - O,5.17028,30.97000,5.17028,1,1,66.11199,1178.69268,Overstock
1968,2022-11-03,HAYWARD,CHICKEN XXXXX - O,22.93990,137.41000,22.93990,1,1,58.57369,155.33543,Overstock
936,2022-07-12,LAKEWOOD,DRUMSTICKS NNNN,442.05055,437.63000,442.05055,1,1,227.21780,-48.59914,Understock
1915,2022-10-28,FREMONT,CHICKEN XXXXX - O,44.05677,263.90000,44.05677,1,1,63.40014,43.90555,Overstock


In [18]:
#conditions = [
#    (data['stock'] == ""),
#    (data['stock_percentage'] > 0) & (data['stock_percentage'] <= 25),
#   (data['stock_percentage'] > 25)]

In [12]:
def stock_flag(row):
    if row["stock"]== 'Overstock':
        return 1
    elif row["stock"]== 'Understock':
        return 2
    if row["stock"]== 'Onhand':
        return 3

In [15]:
data["stock_flag"] = data.apply(stock_flag,axis=1)

In [ ]:
#data['Overstock_flag'] = np.where(data['stock']== 'Overstock', 1, 0)
#data['Understock_flag'] = np.where(data['stock']== 'Understock', 1, 0)
#data['Onhand_flag'] = np.where(data['stock']== 'Onhand', 1, 0)

In [16]:
data.head(10)

,Date,Location,Item,Quantity,Amount,Trend,seasonality,remainder,Forecasted_Quantity,stock_percentage,stock,stock_flag
0,2022-03-10,FREMONT,CHICKEN XXXXX - O,31.71118,189.95000,31.71118,1,1,79.18177,149.69670,Overstock,1
1,2022-03-11,FREMONT,CHICKEN XXXXX - O,53.33056,319.45000,53.33056,1,1,85.55376,60.42164,Overstock,1
2,2022-03-15,RICHMOND CA,FILLET QQQQ,94.19899,752.65000,94.19899,1,1,82.20319,-12.73453,Understock,2
3,2022-03-16,FREMONT,CHICKEN XXXXX - O,84.58932,506.69000,84.58932,1,1,80.07380,-5.33817,Understock,2
4,2022-03-16,RICHMOND CA,FILLET QQQQ,59.15895,472.68000,59.15895,1,1,80.48358,36.04633,Overstock,1
5,2022-03-17,FREMONT,CHICKEN XXXXX - O,45.64942,273.44000,45.64942,1,1,79.62633,74.43010,Overstock,1
6,2022-03-21,RICHMOND CA,CHICKEN XXXXX - O,82.32723,493.14000,82.32723,1,1,88.04430,6.94433,Onhand,3
7,2022-03-22,FREMONT,CHICKEN XXXXX - O,18.45910,110.57000,18.45910,1,1,80.87337,338.12194,Overstock,1
8,2022-03-22,RICHMOND CA,FILLET QQQQ,61.84983,494.18000,61.84983,1,1,76.30180,23.36623,Onhand,3
9,2022-03-22,RICHMOND CA,CHICKEN XXXXX - O,76.22037,456.56000,76.22037,1,1,87.81059,15.20619,Onhand,3


In [19]:
#data.to_excel("Over-Under-Stock.xlsx")